<a href="https://colab.research.google.com/github/luizinfpp/python-notebooks/blob/sentiment-analysis/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Library install

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

#Gathering data

In [15]:
data_neg = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/sentiment/neg_tweets.txt", header=None)
data_pos = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/sentiment/pos_tweets.txt", header=None)